# Import libraries:

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import world_bank_data as wb
from kaleido.scopes.plotly import PlotlyScope

# Load and preprocess data:

In [ ]:
CO2_EMISSION_DATA = "EN.ATM.CO2E.KT"
COUNTRY_POPULATION_SIZE_DATA = "SP.POP.TOTL"
COUNTRY_LAND_AREA_DATA = "AG.LND.TOTL.K2"
COUNTRY_GDP_DATA = "NY.GDP.MKTP.CD"

## CO2 emissions data:

In [ ]:
co2_emissions_df = (
    wb.get_series(CO2_EMISSION_DATA, simplify_index=True)
    .reset_index()
    .rename(columns={CO2_EMISSION_DATA: "co2 emissions"})
)

In [ ]:
co2_emissions_df.head()

## Country population data:

In [ ]:
country_population_df = (
    wb.get_series(COUNTRY_POPULATION_SIZE_DATA, simplify_index=True)
    .reset_index()
    .rename(columns={COUNTRY_POPULATION_SIZE_DATA: "population size"})
)

In [ ]:
country_population_df.head()

## Country land area data:

In [ ]:
# year 2017 has the most recent non-missing data
country_land_area_df = (
    wb.get_series(COUNTRY_LAND_AREA_DATA, simplify_index=True, date="2017")
    .reset_index()
    .rename(columns={COUNTRY_LAND_AREA_DATA: "land area"})
)

In [ ]:
country_land_area_df.head()

## Country GDP data:

In [ ]:
country_gdp_df = (
    wb.get_series(COUNTRY_GDP_DATA, simplify_index=True)
    .reset_index()
    .rename(columns={COUNTRY_GDP_DATA: "GDP"})
)

In [ ]:
country_gdp_df.head()

## Country income group data:

In [ ]:
countries_df = wb.get_countries().reset_index()

In [ ]:
countries_df.head()

## Create aggregated data:

In [ ]:
co2_emissions_agg_df = (
    co2_emissions_df.groupby(by="Country")
    .agg({"co2 emissions": "sum"})
    .reset_index()
    .sort_values(by="co2 emissions", ascending=False)
)

In [ ]:
co2_emissions_agg_df.head()

## Merge data into two data frames:

### Merge not-time based statistics:

In [ ]:
columns_to_drop = [
    "id",
    "iso2Code",
    "name",
    "adminregion",
    "lendingType",
    "capitalCity",
    "longitude",
    "latitude",
]

In [ ]:
overall_stats_df = (
    co2_emissions_agg_df.merge(right=country_land_area_df, on="Country")
    .merge(right=countries_df, left_on="Country", right_on="name")
    .drop(columns=columns_to_drop)
    .sort_values(by="co2 emissions", ascending=False)
)

In [ ]:
overall_stats_df.head()

### Merge time-based data frames:

In [ ]:
time_series_df = (
    co2_emissions_df.merge(right=country_gdp_df, on=["Country", "Year"])
    .merge(right=country_population_df, on=["Country", "Year"])
    .sort_values(by="Year")
)

In [ ]:
time_series_df.head()

## Convert `dtyps`:

### `overall_stats_df`:

In [ ]:
overall_stats_df.dtypes

### `time_series_df`:

In [ ]:
time_series_df.dtypes

In [ ]:
time_series_df = time_series_df.astype({"Year": "int"})

In [ ]:
time_series_df.dtypes

## Drop non-country rows

In [ ]:
non_countries = countries_df.loc[lambda x: x["region"] == "Aggregates"]["name"].values

In [ ]:
non_countries = np.append(non_countries, "Latin America & Caribbean")
non_countries = np.append(non_countries, "Sub-Saharan Africa")

### `overall_stats_df`:

In [ ]:
overall_stats_df.shape

In [ ]:
overall_stats_df.drop(
    index=overall_stats_df[overall_stats_df["Country"].isin(non_countries)].index,
    inplace=True,
)

In [ ]:
overall_stats_df.shape

### `time_series_df`:

In [ ]:
time_series_df.shape

In [ ]:
time_series_df.drop(
    index=time_series_df[time_series_df["Country"].isin(non_countries)].index,
    inplace=True,
)

In [ ]:
time_series_df.shape

## Create new statistics:

In [ ]:
overall_stats_df.head()

### CO2 emissions per square k.m:

In [ ]:
# overall_stats_df["emissions per area"] = (
#     overall_stats_df["co2 emissions"] / overall_stats_df["land area"]
# )

### CO2 emissions per capita:

In [ ]:
time_series_df["emissions per capita"] = (
    time_series_df["co2 emissions"] / time_series_df["population size"]
)

# Data visualization:

In [ ]:
def get_top_n_countries(n: int, by: str):
    return overall_stats_df.sort_values(by=by, ascending=False)[:n]["Country"].values

## Global emissions throught the years:

### Yearly gloabl emissions:

Let's first take a look at CO2 emissions, and how it increased year by year:

In [ ]:
temp_df = (
    time_series_df.groupby(by="Year")
    .agg({"co2 emissions": np.sum})
    .reset_index()
    .sort_values(by="Year")
    .loc[lambda x: x["co2 emissions"] > 0]
)

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="Year",
    y="co2 emissions",
    title="Total CO2 emissions every year",
    labels={"co2 emissions": "CO2 emissions"},
    color="co2 emissions",
    color_continuous_scale=["#FAA307", "#DC2F02"],
)

fig.show()

In [ ]:
scope = PlotlyScope()

In [ ]:
with open("charts/total-co2-emissions-every-year.png", "wb") as f:
    f.write(scope.transform(fig, "png"))

It's obvious that the emissions are increasing every year, not at a constant pace, but the difference between each 10 years is huge.

The emissions are not increasing between each year and the next year, sometimes it's stay the same and sometimes it decreases.

Let's see the difference in total emissions between every consecutive years:

In [ ]:
temp_df["change in emissions"] = temp_df["co2 emissions"].diff()

In [ ]:
temp_df["change in emissions label"] = "increase"

In [ ]:
temp_df.loc[
    lambda x: x["change in emissions"] < 0, "change in emissions label"
] = "decrease"

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="Year",
    y="change in emissions",
    color="change in emissions label",
    title="Change in CO2 emissions every year",
)

fig.update_layout(legend_title_text="change in emissions")

fig.show()

In [ ]:
with open("charts/change-in-co2-emissions-every-year.png", "wb") as f:
    f.write(scope.transform(fig, "png"))

We can see that at some years, the emissions were actually **decerasing**, or increasing at a low pace, but some other years the increase was huge. at year `1970` the total emissions increased by more than one million, and the same for year `1991`, and year `2010` had the biggest increase by nearly two millions.

### Global emissions every decade:

The previous section showed the total global emissions for every single year, but we can see that there's a huge difference in the total emissions now and in the previous 40 years.

Let's see how the change for every decade (10 years) would look like:

In [ ]:
temp_df = (
    time_series_df.groupby(by=time_series_df.Year // 10 * 10)
    .agg({"co2 emissions": np.sum})
    .reset_index()
    .sort_values(by="Year")
    .loc[lambda x: x["co2 emissions"] > 0]
)

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="Year",
    y="co2 emissions",
    title="Total CO2 emissions every decade",
    labels={"co2 emissions": "CO2 emissions"},
    color="co2 emissions",
    color_continuous_scale=["#FAA307", "#DC2F02"],
)

fig.show()

Aggregating the carbon dioxide emissions at the decade level reveal new information: the last decade (2000-2010) had the most global emissions.

Let's study the downward/upward trend in the change of carbon dioxide emissions at the decade level.

In [ ]:
temp_df["change in emissions"] = temp_df["co2 emissions"].diff()

In [ ]:
temp_df["change in emissions label"] = "increase"

In [ ]:
temp_df.loc[
    lambda x: x["change in emissions"] < 0, "change in emissions label"
] = "decrease"

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="Year",
    y="change in emissions",
    color="change in emissions label",
    title="Change in CO2 emissions every decade",
)

fig.update_layout(legend_title_text="Change in emissions")

fig.show()

The biggest increase in CO2 emissions was between the years `1970` and `1980`, during the time the total emissions increased by nearly 60 million.

## Which countries emit the most carbon dioxide today:

The previous section showed how the carbon dioxide emissions were increasing rapidly, and how different the increase was between decades.

Here, in this section, we'll focus more at which *countries* were emitting the most in the last decade (year `2010` to `2016`), then in the next section we'll see the historical emissions, and how that will lead us to different conclusions.

**Note**: the data set contains only values up to year `2016`, that's why later years are missing from the analysis.

In [ ]:
time_series_df.head()

In [ ]:
temp_df = (
    time_series_df[time_series_df.Year >= 2010]
    .groupby(by="Country")
    .agg({"co2 emissions": np.mean})
    .reset_index()
    .sort_values(by="co2 emissions", ascending=False)
    .dropna()
)

In [ ]:
temp_df.head()

In [ ]:
px.bar(
    data_frame=temp_df[:20],
    x="Country",
    y="co2 emissions",
    color="co2 emissions",
    color_continuous_scale=["#FAA307", "#DC2F02"],
    labels={"co2 emissions": "CO2 emissions"},
    title="Top 20 most emitter countries",
)

In [ ]:
threshold = temp_df.iloc[10, 1]

In [ ]:
temp_df.loc[lambda x: x["co2 emissions"] <= threshold, "Country"] = "Rest of the world"

In [ ]:
px.pie(data_frame=temp_df, names="Country", values="co2 emissions", color='Country')